In [4]:
import sys
sys.path.insert(1, '../FeatureEngineering')

import pandas as pd
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.feature_extraction.text import TfidfVectorizer
from hyperparameter_tuning import random_search
from feature_builder import process_dataset
import nltk
import nbimporter
import Exporter as exporter
from hyperparameter_tunning import obtener_hiperparametros

hyper_params = obtener_hiperparametros()

Importing Jupyter notebook from hyperparameter_tunning.ipynb


In [2]:
x_train = pd.read_csv('train.csv')
y_train=x_train[['id','target']]
test=pd.read_csv('test.csv')

In [5]:
#Parametros a usar de manera aleatoria
params = obtener_hiperparametros()['XGBoost']

Utilizo unicamente los features de feature_builder

In [4]:
x_train_features = process_dataset(x_train,use_spacy=True)

Embeddings loaded!
Percentage of words covered in the embeddings = 0.4875485193423176
Embeddings loaded!
Percentage of words covered in the embeddings = 0.5959707770644233


<h3>BUSQUEDA DE PARAMETROS</h3>

In [1]:
xgb = XGBClassifier(objective='binary:logistic',
                    silent=True)

NameError: name 'XGBClassifier' is not defined

In [8]:
random_search(x_train_features,y_train['target'],xgb,params)

Fitting 5 folds for each of 220 candidates, totalling 1100 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:  8.8min
[Parallel(n_jobs=4)]: Done 120 tasks      | elapsed: 41.3min
[Parallel(n_jobs=4)]: Done 280 tasks      | elapsed: 96.4min
[Parallel(n_jobs=4)]: Done 504 tasks      | elapsed: 171.6min
[Parallel(n_jobs=4)]: Done 792 tasks      | elapsed: 272.6min
[Parallel(n_jobs=4)]: Done 1100 out of 1100 | elapsed: 368.3min finished


[08:00:56] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.1.0\src\learner.cc:480: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.



 Time taken: 6 hours 8 minutes and 32.2 seconds.

 Best f1 score with 5-folds and 220 combinations of hyperparameters:
0.7665666582437686

 Best hyperparameters:
{'subsample': 0.9, 'n_estimators': 600, 'max_depth': 3, 'learning_rate': 0.05, 'colsample_bytree': 0.9}


<h3>Prueba local<h3>

In [ ]:
x_train_feat, x_test_feat, y_train_feat, y_test_feat = train_test_split(x_train_features, y_train, test_size = .33, random_state = 17)

In [6]:
xgbooster = XGBClassifier(max_depth=3, n_estimators=600, colsample_bytree=0.9,
                        subsample=0.9, nthread=4, learning_rate=0.05)

In [ ]:
xgbooster.fit(x_train_feat, y_train_feat['target'])

In [10]:
predictions_features = xgbooster.predict(x_test_feat)
accuracy_score(y_test_feat['target'], predictions_features)

0.8209311579785118

In [11]:
f1_score(y_test_feat['target'], predictions_features)

0.7791952894995092

In [ ]:
scores = cross_val_score(xgbooster, x_train_features, y_train['target'], cv=5, scoring='f1_macro')

In [ ]:
print(scores)
print(scores.mean())

<h3>Export<h3>

In [7]:
test_features=process_dataset(test,use_spacy=True)

Embeddings loaded!
Percentage of words covered in the embeddings = 0.5653890824622532
Embeddings loaded!
Percentage of words covered in the embeddings = 0.6646569049669825


In [8]:
exporter.export_model_csv(xgbooster,x_train_features,test_features,test,y_train['target'],'xgb_prueba')